In [1]:
import gradio as gr
import tensorflow as tf
import numpy as np
import os
from PIL import Image

# Use current working directory (not __file__) so this works inside notebooks
BASE_DIR = os.getcwd()
MODEL_PATH = os.path.join(BASE_DIR, 'ai_detector_model.keras')
model = None

def load_model_once():
    global model
    if model is None:
        if not os.path.exists(MODEL_PATH):
            raise FileNotFoundError(f"Model not found at {MODEL_PATH}")
        model = tf.keras.models.load_model(MODEL_PATH)
    return model

def predict_image(image, threshold):
    if image is None:
        return "No Image", {"AI Generated": 0.0, "Real Photo": 0.0}
    img = image.convert('RGB').resize((224, 224))
    arr = np.array(img).astype('float32') / 255.0
    arr = np.expand_dims(arr, axis=0)
    active_model = load_model_once()
    pred = active_model.predict(arr)
    score = float(pred.flatten()[0])
    # calibrated display around the threshold (visual mapping)
    if score < threshold:
        calibrated = 0.5 * (score / threshold) if threshold > 0 else 0.0
        verdict = '📸 REAL PHOTO'
    else:
        calibrated = 0.5 + 0.5 * ((score - threshold) / (1 - threshold)) if threshold < 1 else 1.0
        verdict = '🤖 AI GENERATED'
    calibrated = round(float(calibrated), 3)
    return verdict, {"AI Generated": calibrated, "Real Photo": round(1 - calibrated, 3)}

with gr.Blocks() as demo:
    gr.Markdown('# 🕵️‍♂️ AI vs Real Image Detector (Tunable)')
    with gr.Row():
        with gr.Column():
            input_image = gr.Image(type="pil", label="Upload Image")
            threshold_slider = gr.Slider(minimum=0.1, maximum=0.9, value=0.4, step=0.05, label="AI Sensitivity Threshold")
            submit_btn = gr.Button("Analyze Image")
        with gr.Column():
            output_text = gr.Label(label="Verdict")
            output_conf = gr.Label(label="Confidence")
    submit_btn.click(fn=predict_image, inputs=[input_image, threshold_slider], outputs=[output_text, output_conf])

# Launch only when run as script - in notebooks you can run the last cell to start the app
if __name__ == '__main__':
    print('Starting Gradio demo...')
    demo.launch(share=False)

C:\Users\jaise\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Starting Gradio demo...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
